# IMPORT

In [ ]:
try:
    import google.colab # type: ignore
    colab = True
except:
    colab = False

if colab:
    !git clone "https://github.com/cybernetic-m/eai-project.git" # type: ignore

In [ ]:
# Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR 

# Others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
 
# Our files
if colab:
    sys.path.append('/content/eai-project/training')
    sys.path.append('/content/eai-project/preprocessing')
    sys.path.append('/content/eai-project/dataset')
    sys.path.append('/content/eai-project/utils')
    sys.path.append('/content/eai-project/models')
    sys.path.append('/content/eai-project/testing')
    from train import train
    from preprocessing import *
    from thermal_dataset import thermal_dataset 
    from utils import *
    from complete_model import complete_model 
    from lstm_only import lstm_only
    from testing import test
    prefix = '/content'
        
else:
    from training.train import train
    from preprocessing.preprocessing import *
    from dataset.thermal_dataset import thermal_dataset
    from utils.csv_utils import *
    from models.complete_model import complete_model
    from models.complete_model_autoencoder import complete_model_autoencoder
    from models.lstm_only import lstm_only
    from testing.test import test
    from blocks import mlp, linear, rnn, lstm, cnn
    prefix = '.'
    
pd.set_option('display.max_columns', None)

In [ ]:
hyper_parameters = {'num_epochs': 30,
                    'batch_size': 256,
                    'hidden_dim' : 3,
                    'lr': 0.001,
                    'mode': 'auto-weighted', #average, mlp, linear, auto-weighted
                    'extractor_type': 'autoencoder', #lstm, rnn, autoencoder
                    'ensemble': True,
                    'timesteps': 200,
                    'window_size':30,
                    'norm':True,
                    'file':1,
                    'weight_decay': 0,
                    'lr_multipliers_extractor': 0.75,
                    'lr_multipliers_ensemble': {
                        mlp: 1,
                        linear: 10,
                        rnn: 2,
                        lstm: 1,
                        cnn: 1
                    }

                        }

# Parameters of the convolutional autoencoder
conv_autoencoder_dict = {'in_kern_out': [[4, 3, 5], [5, 3, 6],[6, 3, 7]], # List of hyperparam of autoencoder [[in_channels, kernel_size, out_channels], ...]
                    'pooling_kernel_size': 2, # how much big is the kernel of the pooling (i.e. 2 means halving the dimension each layer)
                    'padding': 'same', # 'same', 'full', 'valid'
                    'pooling': 'max', # 'max' for Max Pooling and 'avg' for Average Pooling 
                    'scale_factor': 2, # upsample scale_factor, 2 means double the dimension each layer
                    'upsample_mode': 'linear' # mode of the upsampling 'linear', 'nearest'
 }

ensemble_model = {'mlp': [{'layer_dim_list': [3,4,3]}],  
              'ARIMA': [{'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1}],
              'linear_regressor': [{'in_features': 3, 'out_features': 3}]
              }

feature_dim = (hyper_parameters['timesteps'] / (2*len(conv_autoencoder_dict['in_kern_out']) )*conv_autoencoder_dict['in_kern_out'][-1][-1])

print(feature_dim)

ensemble_model = {
    'mlp': [{'layer_dim_list': [ 175,int(175/1.5),int(175/3),int(175/5),int(175/8),int(175/10),3]},
            {'layer_dim_list': [ 175,int(175*1.5),3]}],  
    #'linear_regressor': [{'in_features': hyper_parameters['num_layers']*hyper_parameters['hidden_dim'], 'out_features': 3, 'bias':False}],
    'lstm': [{'feature_dim':3, 'input_dim':175, 'num_layers':2}],
    'rnn': [{'feature_dim':3, 'input_dim':175, 'num_layers':4}],
    #'cnn': [{'input_dim':hyper_parameters['hidden_dim'],
    #            'hidden_dim_list': [ int(175/10),int(175/20),int(175/30)],
    #            'output_dim': 3,
    #            'kernel_size_list':[2,4,2]
    #            }]
}

ensemble_model1 = { 
              'ARIMA': [{'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1}],
              'linear_regressor': [{'in_features': 3, 'out_features': 3}, {'in_features': 3, 'out_features': 3}]
              }

ensemble_model2 = {
              'mlp': [{'layer_dim_list': [3,4,3]}],  
              'linear_regressor': [{'in_features': 3, 'out_features': 3}, {'in_features': 3, 'out_features': 3}]
              }

In [ ]:
skip = False
test_flag = True
if os.path.exists(prefix+'/X'+str(hyper_parameters['window_size'])+'testing'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy') and os.path.exists(prefix+'/X'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy'):
    skip = True

**Reproducibility and Device Setting**

In [ ]:
# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# DATASET

In [ ]:
link_zipped_csv = 'https://drive.google.com/file/d/1MssQF4pI_rZqiiDBP4XaLTT1ZaN6ykLm/view?usp=drive_link'
gdrive_link = 'https://drive.google.com/uc?id='
csv_dir = './csv'
zipped_file = './csv.zip'

download_csv(
    link_zipped_csv,
    gdrive_link,
    zipped_file
)

unzip_csv(
    zipped_file,
    csv_dir,
)


In [ ]:
if not skip:

    path = '/content/csv/thermal_drift_features_lab_05_02.csv'

    # Read all the CSV files containing the Temperatures
    features_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_02.csv'))
    features_2 = pd.read_csv(os.path.join(prefix, 'csv/thermal_drift_features_lab_05_03.csv'))
    features_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_04.csv'))
    features_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_05.csv'))
    features_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_06.csv'))

    # Read all the CSV files containing the X1, Y1, Z1 
    targets_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_02.csv'))
    targets_2 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_03.csv'))
    targets_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_04.csv'))
    targets_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_05.csv'))
    targets_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_06.csv'))
    
    # Normalized columns files for X1, Y1, Z1 for the plot
    targets_norm_1= normalize_columns(targets_1, ['X1','Y1','Z1'], -1, 1)
    targets_norm_2= normalize_columns(targets_2, ['X1','Y1','Z1'], -1, 1)
    targets_norm_3= normalize_columns(targets_3, ['X1','Y1','Z1'], -1, 1)
    targets_norm_4= normalize_columns(targets_4, ['X1','Y1','Z1'], -1, 1)
    targets_norm_5= normalize_columns(targets_5, ['X1','Y1','Z1'], -1, 1)
    
    features_norm_1= normalize_columns(features_1, ['Temp Sensor 1','Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4'], -1, 1)
    features_norm_2= normalize_columns(features_2, ['Temp Sensor 1','Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4'], -1, 1)
    features_norm_3= normalize_columns(features_3, ['Temp Sensor 1','Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4'], -1, 1)
    features_norm_4= normalize_columns(features_4, ['Temp Sensor 1','Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4'], -1, 1)
    features_norm_5= normalize_columns(features_5, ['Temp Sensor 1','Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4'], -1, 1)

    # Create a list of features and targets to use for the next drop
    if hyper_parameters['norm']:
        features = [features_norm_1, features_norm_2, features_norm_3, features_norm_4, features_norm_5]
        targets = [targets_norm_1,targets_norm_2,targets_norm_3,targets_norm_4,targets_norm_4]
    else:
        features = [features_1, features_2, features_3, features_4, features_5]
        targets = [targets_1,targets_2,targets_3,targets_4,targets_5]
        
        


In [ ]:
if not skip:
    for feature, target in zip(features, targets):
        feature.drop([
            "name", "tags",
            "2\"Tray1 Vacuum Sensor", "2\"Tray2 Vacuum Sensor", "2\"Tray3 Vacuum Sensor",
            "Avg Oven Temperature", "Chuck Temp [Cdeg]", "Chuck Temp2 [Cdeg]",
            "Chuck1 Vacuum Sensor", "Contrast", "Device State",
            "Dispenser1 Pressure Sensor", "Machine Room Temp", "Main Air", "Main Vacuum",
            "Oven Temperature", "PE_Rx", "PE_Ry", "PE_Rz", "PE_X1", "PE_Y1", "PE_Z1",
            "PUT1 Flow Sensor", "PUT2 Flow Sensor1", "PUT2 Flow Sensor2",
            "PUT2 Flow Sensor3", "PUT2 Flow Sensor4", "PUT2 Flow Sensor5",
            "Photodiode", "Pixel Power", "Preciser1 Vacuum Sensor",
            "Tec FIB1 Holder", "Tec FIB1 Plate", "Tec FIB2 Holder", "Tec FIB2 Plate",
            "Torque11","Torque2","Torque3","Torque4","Torque5","Torque6"
        ], axis=1, inplace=True)
        if 'name' in target.keys() and 'tags' in target.keys():

            target.drop(['name', 'tags'], axis=1, inplace=True)
            
            


In [ ]:
if not skip:
    print(features[1]) # Print the features_1 table

In [ ]:
if not skip:

    print(targets[1]) # Print the target_1

In [ ]:
if not skip:
    # Put X1, Y1, Z1 on the same row of X1 eliminating the NAN values
    fixed_targets = [] # Create a list of target in which we put X1, Y1, Z1 in the same row
    for target in targets:
        fixed_targets.append(transform_dataframe(target)) # iterate over target_1,2,3 ... and append in fixed_targets
        
        

In [ ]:
if not skip:

    print(fixed_targets[1]) # Print the fixed_target_1

In [ ]:
if not skip:
    # Merge of targets with features in one single dataframe
    complete_numbers_list = [] # List of the table with columns that are numbers (0,1,2..) in which we unify both features and targets merging on closest time row
    for fixed_target, feature in zip(fixed_targets, features):
        complete_numbers_list.append(merge_on_closest_time(fixed_target.reset_index(), feature.reset_index()))

    trainig_number_list = []
    testing_number_list = []
    for i in range(len(complete_numbers_list)):
        part_numbers_list = complete_numbers_list[:i] + complete_numbers_list[i+1:]
        trainig_number_list.append(pd.concat(part_numbers_list))
        testing_number_list.append(complete_numbers_list[i])
        
    complete_numbers_dataframe = pd.concat(complete_numbers_list)
    


In [ ]:
if not skip: 
    print(complete_numbers_list) # Print of one example

In [ ]:
if not skip:

    complete = complete_numbers_dataframe.rename(columns={
        0: 'id',
        1: 'time',
        2: 'X1',
        3: 'Y1',
        4: 'Z1',
        5: 'to_remove',
        6: 'time_2',
        7: 'Temp1',
        8: 'Temp2',
        9: 'Temp3',
        10: 'Temp4'
        })
    complete.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
    training_list = []
    testing_list = []
    for training, testing in zip(trainig_number_list, testing_number_list):
        training_tmp = training.rename(columns={
            0: 'id',
            1: 'time',
            2: 'X1',
            3: 'Y1',
            4: 'Z1',
            5: 'to_remove',
            6: 'time_2',
            7: 'Temp1',
            8: 'Temp2',
            9: 'Temp3',
            10: 'Temp4'
            })
        training_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
        training_list.append(training_tmp)
        
        testing_tmp = testing.rename(columns={
            0: 'id',
            1: 'time',
            2: 'X1',
            3: 'Y1',
            4: 'Z1',
            5: 'to_remove',
            6: 'time_2',
            7: 'Temp1',
            8: 'Temp2',
            9: 'Temp3',
            10: 'Temp4'
            })
        testing_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
        testing_list.append(testing_tmp)
        

In [ ]:
if not skip:

    print(training_list)

In [ ]:
if not skip:
    '''
    complete.astype(float)

    X = complete[['Temp1','Temp2', 'Temp3', 'Temp4']]
    Y = complete[['X1', 'Y1', 'Z1']]

    Y = Y.values.astype(np.float32)
    Y = my_gradient(Y, window_size=hyper_parameters['window_size'])
    
     
    X = X.values.astype(np.float32)
    X /= 100
    X = X[len(X):20]
    '''
    print(hyper_parameters['window_size'])
    # Save the features and targets in file npy
    #np.save(prefix+'/X'+str(hyper_parameters['window_size'])+'.npy',X)
    #np.save(prefix+'/Y'+str(hyper_parameters['window_size'])+'.npy',Y)
    for i, (testing, training) in enumerate(zip(testing_list, training_list)):
        trainingx = normalize_columns(training, ['Temp1','Temp2', 'Temp3', 'Temp4'],-1,1)
        trainingy = normalize_columns(training, ['X1', 'Y1', 'Z1'],-1,1)
        training.astype(float)
        #print(training)
        X = trainingx[['Temp1','Temp2', 'Temp3', 'Temp4']]
        Y = trainingy[['X1', 'Y1', 'Z1']]
        
        Y = Y.values.astype(np.float32)
        Y = my_gradient(Y, window_size=hyper_parameters['window_size'])
        Y = normalize_array(Y,-1,1)
        
        X = X.values.astype(np.float32)
        print('training'+str(i))
        print('X',len(X))
        print(np.max(X))
        print('Y',len(Y))
        print(np.max(Y[:,0]))
        print(np.max(Y[:,1]))
        print(np.max(Y[:,2]))
        # Save the features and targets in file npy
        np.save(prefix+'/X'+str(hyper_parameters['window_size'])+'training'+str(i)+str(hyper_parameters['norm'])+'.npy',X)
        np.save(prefix+'/Y'+str(hyper_parameters['window_size'])+'training'+str(i)+str(hyper_parameters['norm'])+'.npy',Y)
        testingx = normalize_columns(testing, ['Temp1','Temp2', 'Temp3', 'Temp4'],-1,1)
        testingy = normalize_columns(testing, ['X1', 'Y1', 'Z1'],-1,1)
        testing.astype(float)

        X = testingx[['Temp1','Temp2', 'Temp3', 'Temp4']]
        Y = testingy[['X1', 'Y1', 'Z1']]

        Y = Y.values.astype(np.float32)
        Y = my_gradient(Y, window_size=hyper_parameters['window_size'])
        Y = normalize_array(Y,-1,1)
        
        X = X.values.astype(np.float32)
        print('testing'+str(i))
        print('X',len(X))
        print(np.max(X))
        print('Y',len(Y))
        print(np.max(Y))     
        # Save the features and targets in file npy
        np.save(prefix+'/X'+str(hyper_parameters['window_size'])+'testing'+str(i)+str(hyper_parameters['norm'])+'.npy',X)
        np.save(prefix+'/Y'+str(hyper_parameters['window_size'])+'testing'+str(i)+str(hyper_parameters['norm'])+'.npy',Y)


In [ ]:
if not skip:

    targets_norm_1['time'] = pd.to_datetime(targets_norm_1['time'], unit='ns')

    # Reshape the DataFrame using melt()
    targets_melted = targets_norm_1.reset_index().melt(id_vars=['time'], value_vars=['X1', 'Y1', 'Z1'], var_name='variable', value_name='value')

    # Drop rows where 'value' is NaN (to keep only the non-null entries)
    targets_melted = targets_melted.dropna(subset=['value'])

    # Plot the data
    plt.figure(figsize=(10, 6))
    for var in targets_melted['variable'].unique():
        # Filter data for each variable and plot
        temp_df = targets_melted[targets_melted['variable'] == var]
        plt.plot(temp_df['time'], temp_df['value'], label=var)

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of X1, Y1, Z1 over Time')
    plt.legend(title='Variable')

    # Show the plot
    plt.show()

    #targets.plot(y='X1',x='time')

In [ ]:
if not skip:

    # Convert 'time' to datetime (nanoseconds to datetime)
    features_1['time'] = pd.to_datetime(features_1['time'], unit='ns')

    # Set 'time' as the index
    features_1.set_index('time', inplace=True)

    # Optionally, you can plot X1, Y1, Z1 directly
    features_1[['Temp Sensor 1', 'Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4', ]].plot()

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of Temp Sensor 1, Temp Sensor 2, Temp Sensor 3, Temp Sensor 4 over Time')

    # Show the plot
    plt.show()

In [ ]:
if skip:
    # If skip is true, we simply load the data 
    X = np.load(prefix+'/X'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+'.npy')
    Y = np.load(prefix+'/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+'.npy')

print(X)
print(Y)

splitPerc = [0.7,0.3]
splitted_X = split(X, splitPerc)
splitted_Y = split(Y, splitPerc)

In [ ]:
for i in [15, 20, 25]:
    X = np.load(prefix+'/X'+str(i)+'.npy')
    Y = np.load(prefix+'/Y'+str(i)+'.npy')

    print(X)
    print(Y)

# MODEL

In [ ]:
datasetTrain = thermal_dataset((splitted_X[0],splitted_Y[0]), hyper_parameters['timesteps'], device)
datasetVal = thermal_dataset((splitted_X[1],splitted_Y[1]), hyper_parameters['timesteps'], device)

In [ ]:
# Training, Test and Validation Dataloader initialization
loaderTrain = DataLoader(datasetTrain, shuffle=True, batch_size=hyper_parameters['batch_size'])
loaderVal = DataLoader(datasetVal, shuffle=True, batch_size=hyper_parameters['batch_size'])

# Model Initialization (True if you want to use the Ensemble model, False in you want to use a single LSTM model)
if hyper_parameters['ensemble'] == False:
    model = lstm_only(hidden_dim=hyper_parameters['hidden_dim'],
                      timesteps=hyper_parameters['timesteps'] ,
                      input_dim=4, 
                      output_dim=3).to(device)
    
elif hyper_parameters['ensemble'] == True:
    if hyper_parameters['extractor_type'] == 'autoencoder':
        model = complete_model_autoencoder(ensemble_model, 
                                           device, 
                                           in_kern_out=conv_autoencoder_dict['in_kern_out'], 
                                           pooling_kernel_size=conv_autoencoder_dict['pooling_kernel_size'], 
                                           padding=conv_autoencoder_dict['padding'], 
                                           pooling=conv_autoencoder_dict['pooling'], 
                                           scale_factor = conv_autoencoder_dict['scale_factor'], 
                                           upsample_mode=conv_autoencoder_dict['upsample_mode'], 
                                           mode='auto-weighted'
                                           ).to(device)
    else:
        model = complete_model(hidden_dim=hyper_parameters['hidden_dim'], 
                               input_dim=4, 
                               model_dict=ensemble_model, 
                               device=device, 
                               mode=hyper_parameters['mode'], 
                               extractor_type=hyper_parameters['extractor_type']).to(device)

# Definition of the optimizer and loss function

if hyper_parameters['ensemble'] == False:
    optimizer = optim.Adam(model.parameters(), lr=hyper_parameters['lr'])
elif hyper_parameters['ensemble'] == True:


    models, arima, rnn_ = model.get_models()
    list_models = models + arima + rnn_ 
    optimizer = [optim.Adam(model.extractor.parameters(), lr=hyper_parameters['lr']*hyper_parameters['lr_multipliers_extractor'])]
    for sub_model in list_models: 
        for model_type, multiplier in hyper_parameters['lr_multipliers_ensemble'].items():
            if isinstance(sub_model, model_type):
                optimizer.append(optim.Adam(model.parameters(), lr=hyper_parameters['lr']*multiplier, weight_decay=hyper_parameters['weight_decay']))
                break
    print(len(optimizer))

hyper_parameters['lr_multipliers_ensemble'] = {
        'mlp': 1,
        'linear': 10,
        'rnn': 2,
        'lstm': 1.5,
        'cnn': 1
    }


loss_fn = nn.MSELoss()
if hyper_parameters['ensemble'] == False:
    scheduler = ExponentialLR(optimizer, gamma=0.85)
elif hyper_parameters['ensemble'] == True:
    scheduler = [ExponentialLR(opti, gamma=0.85) for opti in optimizer]
model

In [ ]:

train(
    num_epochs=hyper_parameters['num_epochs'],
    loss_fn=loss_fn,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    training_dataloader=loaderTrain,
    validation_dataloader=loaderVal,
    hyperparams=hyper_parameters,
    model_dict = ensemble_model,
    autoencoder_dict=conv_autoencoder_dict,
    complete=hyper_parameters['ensemble']
)


Model saved to results/training_2025-02-20_14-01/model.pt
weights of the ensemble models: [7.032132653961298e-10, 0.00757095729932189, 0.8439257740974426, 0.14850331842899323]
train MODEL: LOSS 0.000054830666 MAE X1:0.0017, Y1:0.0035, Z1:0.0032 R2 X1:-0.0025, Y1:-0.0013, Z1:-0.0024 RMSE X1:0.002601, Y1:0.011560, Z1:0.004832
train AUTOENCODER: LOSS 6.800214644401 MAE X1:1.1322, Y1:1.1870, Z1:1.1587 R2 X1:-118.9557, Y1:-110.0853, Z1:-139.2194 RMSE X1:1.134271, Y1:1.192086, Z1:1.160319
valid MODEL: LOSS 0.000009980359 MAE X1:0.0016, Y1:0.0027, Z1:0.0025 R2 X1:-0.0022, Y1:-0.0094, Z1:-0.0153 RMSE X1:0.002058, Y1:0.003600, Z1:0.003380
valid AUTOENCODER: LOSS 6.190421783007 MAE X1:1.0956, Y1:1.1289, Z1:1.1328 R2 X1:-151.8723, Y1:-126.0516, Z1:-185.3826 RMSE X1:1.096094, Y1:1.132995, Z1:1.133307
EPOCH 4/30:
Model saved to results/training_2025-02-20_14-01/model.pt
weights of the ensemble models: [0.001619504066184163, 0.6412848234176636, 0.10419602692127228, 0.2528996467590332]
train MODEL: L

# Test

In [ ]:
# Need to specify a model.pt file to load! In the directory results that you want to test
result_path = '/results/training_2025-01-31_10-13' 
model_path = prefix + result_path +'/model.pt'

# Read hyperparameters
with open (prefix + result_path + '/hyperparam.json', 'r') as f:
    hyperparams = json.load(f)
    
X = np.load(prefix+'/X'+str(hyper_parameters['window_size'])+'testing'+str(hyper_parameters['file'])+'.npy')
Y = np.load(prefix+'/X'+str(hyper_parameters['window_size'])+'testing'+str(hyper_parameters['file'])+'.npy')

datasetTest = thermal_dataset((X,Y), hyperparams['timesteps'], device)
loaderTest = DataLoader(datasetTest, shuffle=True, batch_size=hyperparams['batch_size'])
# Section in which we reinitialize the model for testing depending on the hyperparameters.json and ensemble.json (if present) in the directories of results
# Read Ensemble models data (if complete = True)
if hyperparams['ensemble'] == True:
    # Read Ensemble model parameters and initialize the model
    with open (prefix + result_path + '/ensemble.json', 'r') as f:
        model_dict = json.load(f)
    
    model = complete_model(hidden_dim=hyperparams['hidden_dim'], input_dim=4, model_dict=model_dict, device=device, mode=hyperparams['mode'], extractor_type=hyperparams['extractor_type']).to(device)
# Otherwise create an LSTM only model
else:
    model = lstm_only(hidden_dim=hyperparams['hidden_dim'], timesteps=hyperparams['timesteps'] ,input_dim=4, output_dim=3).to(device)

if test_flag:
    test_metrics, test_loss_avg, total_inference_time, inference_time_avg, y_true_list, y_pred_list = test(
        model=model,
        model_path=model_path,
        test_dataloader=loaderTest,
        loss_fn=loss_fn,
        complete=hyperparams['ensemble']
    )
print("Test metrics:")
print(f"rmse: {test_metrics['rmse']} , mae: {test_metrics['mae']}, r2: {test_metrics['r2']}")
print("Others:")
print(f"Average Loss: {test_loss_avg}, Total Inference Time (All Dataset): {total_inference_time}, Average Inference Time: {inference_time_avg}")


In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyper_parameters['num_epochs']+1)]

# Compute training loss as the square of rmse for each element because loss = mse
# Both for training and validation
training_loss = [rmse**2 for rmse in train_metrics['rmse']] 
valid_loss = [rmse**2 for rmse in valid_metrics['rmse']] 

plt.plot(epochs, training_loss, label='Training', color='b')
plt.plot(epochs, valid_loss, label='Validation', color='r')
plt.title('Loss (RMSE^2)')
plt.xlabel('Epochs')
plt.ylabel('Loss value')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
plt.savefig(path+'/loss')  # Save the figure to the specified path
plt.show()

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyper_parameters['num_epochs']+1)]

plt.plot(epochs, train_metrics['mae'], label='Training', color='b')
plt.plot(epochs, valid_metrics['mae'], label='Validation', color='r')
plt.title('Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE value')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
plt.savefig(path + '/mae')  # Save the figure to the specified path
plt.show()

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyper_parameters['num_epochs']+1)]

plt.plot(epochs, train_metrics['r2'], label='Training', color='b')
plt.plot(epochs, valid_metrics['r2'], label='Validation', color='r')
plt.title('R2 (Coefficient of determination)')
plt.xlabel('Epochs')
plt.ylabel('R2 value')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
plt.savefig(path + '/r2')  # Save the figure to the specified path
plt.show()

In [ ]:
# Create the lists of X1, Y1, Z1 true values to plot
x1_true = [x[0] for x in y_true_list]
y1_true = [y[1] for y in y_true_list]
z1_true = [z[2] for z in y_true_list]

# Create the lists of X1, Y1, Z1 predicted values to plot
x1_pred = [x[0] for x in y_pred_list]
y1_pred = [y[1] for y in y_pred_list]
z1_pred = [z[2] for z in y_pred_list]

# Create x-axis
t = [i for i in range(1, len(y_pred_list)+1)]


In [ ]:
# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
#t = [i for i in range(1, len(Y[:,0])+1)]
plt.plot(t, y1_true, label='Ground Truth', color='b')
plt.plot(t, x1_pred, label='Prediction', color='r')
plt.title('X1 Sequence')
plt.xlabel('Time')
plt.ylabel('X1')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
#plt.savefig(path + '/r2')  # Save the figure to the specified path
plt.show()

In [ ]:
# Initialize a list of epochs [1,2,3, ...] for the plots x-axis

plt.plot(t[:100], y1_true[:100], label='Ground Truth', color='b')
plt.plot(t[:100], y1_pred[:100], label='Prediction', color='r')
plt.title('Y1 Sequence')
plt.xlabel('Time')
plt.ylabel('Y1')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
#plt.savefig(path + '/r2')  # Save the figure to the specified path
plt.show()

In [ ]:
# Initialize a list of epochs [1,2,3, ...] for the plots x-axis

plt.plot(t, z1_true, label='Ground Truth', color='b')
plt.plot(t, z1_pred, label='Prediction', color='r')
plt.title('Z1 Sequence')
plt.xlabel('Time')
plt.ylabel('Z1')
plt.legend()

plt.tight_layout()  # This helps to prevent overlapping of subplots
#plt.savefig(path + '/r2')  # Save the figure to the specified path
plt.show()